## Imports

In [9]:


# General imports
import time
import json
import random
import os

# Intent recognition imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from gpt import GPT
from intent_model import IntentModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
# numpy.random.seed(SEED)
from transformers import set_seed
set_seed(SEED)

## Load data

In [11]:
# Load the data from JSON file
with open("../intent_recognition_v2.json", "r") as f:
    data = json.load(f)

validation_data = []
train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    validation = datum["test_questions"]
    for text in texts:
        train_data.append((text, label))
    for text in validation:
        validation_data.append((text, label))

# Concatenate train and validation data for generating label_map
all_data = train_data + validation_data

# Define the mapping between top-level labels and integers
# Sort the labels before enumerating
label_map = {label: i for i, label in enumerate(sorted(set([data[1] for data in all_data])))}

# Convert the training data labels to integers using the label_map
train_labels = torch.tensor([label_map[data[1]] for data in train_data])
validation_labels = torch.tensor([label_map[data[1]] for data in validation_data])

print(validation_labels)
intent_model = IntentModel(train_data, validation_data, label_map, train_labels, label_map, validation_labels)


tensor([15, 15, 15, 15,  6,  6,  6,  6,  6, 16, 16, 16, 16, 16,  5,  5,  5,  5,
         5, 11, 11, 11,  7,  7,  7,  7,  7,  7,  7,  0,  0,  0,  0,  8,  8,  8,
         8,  8,  3,  3,  3,  3,  3, 12, 12, 12, 12, 12, 12, 10, 10, 10, 10, 10,
        10,  4,  4,  4,  4,  4,  4,  2,  2,  2,  2,  2,  2,  9,  9,  9,  9,  9,
         9,  9,  9, 14, 14, 14, 14, 14, 18, 18, 18, 18, 18, 17, 17, 17, 17, 17,
        17,  1,  1,  1,  1,  1,  1, 13, 13, 13, 13, 13, 13])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1, Training Loss: 3.0333914756774902, Validation Loss: 3.017672538757324, Accuracy: 1
Epoch 2, Training Loss: 2.976868152618408, Validation Loss: 2.8822429180145264, Accuracy: 5
Epoch 3, Training Loss: 2.925187826156616, Validation Loss: 3.090611219406128, Accuracy: 3
Epoch 4, Training Loss: 2.769347667694092, Validation Loss: 3.050124168395996, Accuracy: 3
Epoch 5, Training Loss: 2.6361773014068604, Validation Loss: 3.1885697841644287, Accuracy: 2
Epoch 6, Training Loss: 2.5719809532165527, Validation Loss: 3.3436801433563232, Accuracy: 4
Epoch 7, Training Loss: 2.110271692276001, Validation Loss: 3.1232705116271973, Accuracy: 0
Epoch 8, Training Loss: 1.7517575025558472, Validation Loss: 3.376723289489746, Accuracy: 11
Epoch 9, Training Loss: 1.3497637510299683, Validation Loss: 4.075252056121826, Accuracy: 0
Epoch 10, Training Loss: 0.9746473431587219, Validation Loss: 4.01334285736084, Accuracy: 0
Epoch 11, Training Loss: 0.715197741985321, Validation Loss: 4.398578643798828,

## Load models

In [12]:
print(validation_labels)
intent_model = IntentModel(train_data, validation_data, label_map, train_labels, label_map, validation_labels)


tensor([15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15])


## Check data completeness

In [13]:
from collections import defaultdict

# Your data is already a list of dictionaries
categories = defaultdict(list)

for item in data:
    intent_text = item['intent']
    if not item['use_gpt'] and not item['responses']:

        intent_text += ' (Needs answers)'
    categories[item['category']].append(intent_text)

for category, intents in categories.items():
    # if(category == 'privacy_policy' or category ==  'legal_statement'):
    #     continue

    print(f'Category: {category}')
    for intent in intents:
        print(f'  Intent: {intent}')
    print('\n')

Category: privacy_policy
  Intent: summarization_privacy_policy
  Intent: data_sharing_opt_out
  Intent: third_parties
  Intent: data_collection
  Intent: personal_information
  Intent: data_storage
  Intent: access_personal_data
  Intent: delete_personal_information
  Intent: cookie_policy
  Intent: privacy_policy_updates
  Intent: minimum_age
  Intent: data_breaches
  Intent: contact_privacy_concerns


Category: legal_statement
  Intent: legal_statement_information
  Intent: summarization_legal_statement
  Intent: use_restrictions_legal_statement
  Intent: transfer_legal_statement
  Intent: cancellation_legal_statement
  Intent: refunds_legal_statement




## Get evaluation data

In [14]:
evaluation_file = "./evaluation_results.json"

if os.path.isfile(evaluation_file):
    with open(evaluation_file, "r") as file:
        accuracies = json.load(file)
else:
    accuracies = intent_model.evaluate_model(data)

Question: Could you provide a summary of your privacy policy?, Predicted intent: summarization_privacy_policy, correct intent: summarization_privacy_policy
Question: What are the main points covered in your privacy policy?, Predicted intent: summarization_privacy_policy, correct intent: summarization_privacy_policy
Question: What are the key points of your privacy policy?, Predicted intent: summarization_privacy_policy, correct intent: summarization_privacy_policy
Question: Can you give me a short description of what your privacy policy includes?, Predicted intent: summarization_privacy_policy, correct intent: summarization_privacy_policy
Question: In a few sentences, what does your privacy policy say?, Predicted intent: summarization_privacy_policy, correct intent: summarization_privacy_policy
Question: Could you provide an overview of your privacy policy?, Predicted intent: summarization_privacy_policy, correct intent: summarization_privacy_policy
Question: What is your privacy polic

## Check data correctness

In [15]:
print("total correctness", accuracies["total_correctness"])

# "total_correct_counts": 608,
# "total_counts": 663,

for evaluation in accuracies["results"]:
    if (evaluation["total_correctness"] == 100):
        continue

    print(f"Intent: {evaluation['correct_intent']}")
    print(f"Train question results: {evaluation['train_questions_results']}")

    test_results = evaluation.get('test_questions_results')
    if test_results:
        print(f"Test question results: {test_results}")
        
    advanced_results = evaluation.get('test_questions_advanced_results')
    if advanced_results:
        print(f"Advanced test question results: {advanced_results}")
    
    print(f"Total correctness for all questions: {evaluation['total_correctness']}%")
    print("\n")


total correctness 95.2
Intent: data_sharing_opt_out
Train question results: 19 out of 19 correct
Test question results: 4 out of 5 correct
Advanced test question results: 4 out of 5 correct
Total correctness for all questions: 93.1%


Intent: third_parties
Train question results: 20 out of 20 correct
Test question results: 4 out of 5 correct
Advanced test question results: 4 out of 5 correct
Total correctness for all questions: 93.33%


Intent: data_collection
Train question results: 18 out of 19 correct
Test question results: 4 out of 5 correct
Advanced test question results: 4 out of 5 correct
Total correctness for all questions: 89.66%


Intent: personal_information
Train question results: 18 out of 20 correct
Test question results: 3 out of 3 correct
Advanced test question results: 3 out of 3 correct
Total correctness for all questions: 92.31%


Intent: data_storage
Train question results: 20 out of 20 correct
Test question results: 5 out of 7 correct
Advanced test question results

In [16]:
print(intent_model.get_intent("Can you provide details on the rules pertaining to the age limit for using the company's services?")[0], ": minimum age")
print(intent_model.get_intent("Could you summarize the main provisions of the legal statement?")[0], ": legal_statement_information")
print(intent_model.get_intent("Are you allowed to transfer ownership of my account or subscription to another individual?")[0], ": transfer_legal_statement")
print('test')

minimum_age : minimum age
legal_statement_information : legal_statement_information
transfer_legal_statement : transfer_legal_statement
test
